In [ ]:
!pip install rdkit
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 43.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 KB 45.4 MB/s eta 0:00:00


In [ ]:
import os
import glob
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected

In [ ]:
!wget https://pdbbind.oss-cn-hangzhou.aliyuncs.com/download/PDBbind_v2020_refined.tar.gz

--2023-04-01 14:40:04--  https://pdbbind.oss-cn-hangzhou.aliyuncs.com/download/PDBbind_v2020_refined.tar.gz
Resolving pdbbind.oss-cn-hangzhou.aliyuncs.com (pdbbind.oss-cn-hangzhou.aliyuncs.com)... 121.199.204.182
Connecting to pdbbind.oss-cn-hangzhou.aliyuncs.com (pdbbind.oss-cn-hangzhou.aliyuncs.com)|121.199.204.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 690895072 (659M) [application/gzip]
Saving to: ‘PDBbind_v2020_refined.tar.gz.1’

PDBbind_v2020_refin 100%[===================>] 658.89M  8.16MB/s    in 90s     

2023-04-01 14:41:37 (7.29 MB/s) - ‘PDBbind_v2020_refined.tar.gz.1’ saved [690895072/690895072]



In [ ]:
!mkdir PDBbind_v2020_refined

In [ ]:
!tar -xzvf "PDBbind_v2020_refined.tar.gz" -C "PDBbind_v2020_refined"

In [ ]:
ls /content/PDBbind_v2020_refined/refined-set | wc -l

5318


In [ ]:
!rm -r /content/PDBbind_v2020_refined/refined-set

In [ ]:
%cd ~/../content/

/content


In [ ]:
!ls

Berberin.mol2  INDEX_refined_data.2020	PDBbind_v2020_refined.tar.gz
CDK1.pdb       PDBbind_v2020_refined	sample_data


In [ ]:
import re

# 1. Прочитайте файл и извлеките информацию о связывающей аффинности для каждого идентификатора PDB
affinity_data_file = "INDEX_refined_data.2020"
affinity_data = {}
data_folder = '/content/PDBbind_v2020_refined/refined-set'


with open(affinity_data_file, "r") as f:
    lines = f.readlines()
    for line in lines:
      if 'Kd=' in line:
        tokens = line.split()
        for token in tokens:
              if 'Kd' in token:
                value, unit = re.findall(r"[-+]?\d*\.\d+|\d+", token)[0], token[-2]
                value = float(value)
                  
                if unit == 'u':
                    value *= 1000  # convert from uM to nM
                elif unit == 'm':
                    value *= 1000000  # convert from mM to nM
                elif unit == 'p':
                    value *= 0.001  # convert from mM to nM
                  
                pdb_id = line.split()[0]
                affinity = value
                affinity_data[pdb_id] = value

# 2. Сопоставьте идентификаторы PDB с соответствующими значениями связывающей аффинности
binding_affinities = []
for pdb_id in os.listdir(data_folder):
    if pdb_id in affinity_data:
        binding_affinities.append((pdb_id, affinity_data[pdb_id]))
    else:
        print(f"No affinity data found for {pdb_id}")

binding_affinities_dict = {pdb_id: affinity for pdb_id, affinity in binding_affinities}

No affinity data found for 5er1
No affinity data found for 3cf8
No affinity data found for 5e2r
No affinity data found for 1qf0
No affinity data found for 3ckp
No affinity data found for 4nuc
No affinity data found for 2pou
No affinity data found for 2wb5
No affinity data found for 3tu7
No affinity data found for 6md8
No affinity data found for 4mss
No affinity data found for 3b7j
No affinity data found for 5n0d
No affinity data found for 1w3k
No affinity data found for 4zx4
No affinity data found for 6dpt
No affinity data found for 4gqr
No affinity data found for 3eft
No affinity data found for 6f28
No affinity data found for 3vhk
No affinity data found for 1o33
No affinity data found for 3uw5
No affinity data found for 6ebe
No affinity data found for 1s39
No affinity data found for 6p3t
No affinity data found for 2ojg
No affinity data found for 2bes
No affinity data found for 3qt6
No affinity data found for 1syh
No affinity data found for 4q7w
No affinity data found for 3fuz
No affin

In [ ]:
print(binding_affinities_dict)
print(len(binding_affinities_dict))
values = list(binding_affinities_dict.values())
print(values)
#разыне значения um, микро, наномоли

{'2gzl': 15000.0, '3ebl': 500.0, '3ip5': 600.0, '6pl1': 22.2, '2vo4': 11600.0, '6dzx': 3500.0, '3vhc': 1.3, '4x50': 17.7, '3evd': 124.0, '5tpx': 280.0, '6e7r': 147.0, '3jzj': 248000.0, '6fo5': 190.0, '4r4o': 1500.0, '3uo4': 299.0, '5dx4': 137.0, '6e7t': 129.0, '4zv2': 1170000.0, '5vd2': 78.7, '1b6h': 15.0, '4qpd': 15.0, '2jdp': 250.0, '4n7m': 3400.0, '5d3x': 440.0, '4f7v': 4160.0, '6rnt': 4300000.0, '3lzz': 146000.0, '4pfu': 670.0, '5czm': 4.8, '5ia4': 52.0, '1a99': 2000.0, '4uye': 9.54, '4rlw': 13400.0, '2dri': 130.0, '6cvf': 450.0, '1nw5': 6100.0, '5fs5': 206.4, '2yaz': 7300.0, '6gnp': 28410.0, '4uac': 288.0, '3d7k': 25000.0, '3elc': 2020000.0, '2zkj': 3300.0, '4ad2': 625.0, '5ufr': 2.0, '5d3t': 12000.0, '4or4': 26800.0, '1fao': 43.0, '4og3': 285.0, '3m40': 170.0, '4dkp': 1900.0, '6im4': 40.0, '3e12': 1020.0, '3ujc': 26000.0, '6eln': 900.0, '2fqy': 270.0, '2cex': 20000.0, '4fcq': 101.0, '1d7i': 250000.0, '3qbc': 10800.0, '1g1d': 0.36, '6cks': 15.0, '5fou': 113000.0, '1rmz': 10.0, '6h

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

values = list(binding_affinities_dict.values())
values_np = np.array(values).reshape(-1,1)

scaled_values = scaler.fit_transform(values_np)

for i, key in enumerate(binding_affinities_dict):
    binding_affinities_dict[key] = scaled_values[i][0]

In [ ]:
values = list(binding_affinities_dict.values())
print(values)

[0.0014851483566709147, 4.950479208705027e-05, 5.9405782187628644e-05, 2.19786138648679e-06, 0.0011485146932512498, 0.00034653449510440155, 1.2855445546590964e-07, 1.7523168319607633e-06, 1.2277069308875579e-05, 2.7722613865777844e-05, 1.4554297032008603e-05, 0.02455445529101853, 1.8811722775257305e-05, 0.000148514693092834, 2.9603801984887734e-05, 1.3564198021950767e-05, 1.2772118813904496e-05, 0.11584158401835117, 7.791920793313573e-06, 1.4849900992451472e-06, 1.4849900992451472e-06, 2.4752316835604332e-05, 0.0003366335050038232, 4.356419802670324e-05, 0.0004118810297682189, 0.42574257416645434, 0.014455445388428587, 6.63364752580335e-05, 4.7508910898615276e-07, 5.148356436459146e-06, 0.0001980196435957259, 9.443960397535678e-07, 0.0013267325150616606, 1.2871128714910281e-05, 4.455429703676108e-05, 0.0006039602377194393, 2.043548515175216e-05, 0.0007227721189263797, 0.002812871129158475, 2.8514693073824113e-05, 0.0024752473667287523, 0.19999999987326736, 0.0003267325149032448, 6.1881

In [ ]:
def mol_to_graph(mol):
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append(atom.GetAtomicNum())
    atom_features = torch.tensor(atom_features, dtype=torch.float).view(-1, 1)

    bond_features = []
    edge_index = []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index.extend([[i, j], [j, i]])
        bond_features.extend([bond.GetBondTypeAsDouble()] * 2)
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    bond_features = torch.tensor(bond_features, dtype=torch.float).view(-1, 1)

    graph = Data(x=atom_features, edge_index=edge_index, edge_attr=bond_features)
    return graph

def read_receptor_pdb(path):
    mol = Chem.MolFromPDBFile(path)
    if mol is None:
        raise ValueError(f"Failed to read PDB file: {path}")
    return mol

def read_ligand_mol2(path):
    mol = Chem.MolFromMol2File(path)
    if mol is None:
        raise ValueError(f"Failed to read MOL2 file: {path}")
    return mol

data_folder = '/content/PDBbind_v2020_refined/refined-set'
graph_data = []
skipped_files = 0
counter = 0

for pdb_id in os.listdir(data_folder):
    receptor_path = os.path.join(data_folder, pdb_id, f"{pdb_id}_protein.pdb")
    ligand_path = os.path.join(data_folder, pdb_id, f"{pdb_id}_ligand.mol2")

    if os.path.isfile(receptor_path) and os.path.isfile(ligand_path):
        try:
            receptor_mol = read_receptor_pdb(receptor_path)
            ligand_mol = read_ligand_mol2(ligand_path)

            receptor_graph = mol_to_graph(receptor_mol)
            ligand_graph = mol_to_graph(ligand_mol)

            graph_data.append((pdb_id, receptor_graph, ligand_graph))
            counter += 1
            print(f"Processing: {counter}")
        except Exception as e:
            print(f"Error processing {pdb_id}: {e}")
            skipped_files += 1

print(f"Processed {len(graph_data)} receptor-ligand pairs.")
print(f"Skipped {skipped_files} due to errors.")

Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5
Processing: 6
Processing: 7
Processing: 8
Processing: 9
Processing: 10
Processing: 11
Processing: 12
Processing: 13
Processing: 14
Processing: 15
Processing: 16
Processing: 17
Processing: 18


[14:48:25] Explicit valence for atom # 520 O, 3, is greater than permitted


Error processing 2wb5: Failed to read PDB file: /content/PDBbind_v2020_refined/refined-set/2wb5/2wb5_protein.pdb
Processing: 19
Processing: 20
Processing: 21
Processing: 22
Processing: 23
Processing: 24
Processing: 25
Processing: 26
Processing: 27
Processing: 28
Processing: 29
Processing: 30
Processing: 31
Processing: 32
Processing: 33
Processing: 34
Processing: 35
Processing: 36
Processing: 37
Processing: 38
Processing: 39
Processing: 40
Processing: 41
Processing: 42
Processing: 43
Processing: 44
Processing: 45
Processing: 46
Processing: 47
Processing: 48
Processing: 49
Processing: 50
Processing: 51
Processing: 52
Processing: 53
Processing: 54
Processing: 55
Processing: 56
Processing: 57
Processing: 58
Processing: 59
Processing: 60
Processing: 61
Processing: 62
Processing: 63
Processing: 64
Processing: 65
Processing: 66
Processing: 67
Processing: 68
Processing: 69
Processing: 70
Processing: 71
Processing: 72
Processing: 73
Processing: 74
Processing: 75
Processing: 76
Processing: 77
Pr

[14:57:27] 4ew2_ligand: warning - charged amidine and isFixed atom.


Processing: 170
Processing: 171
Processing: 172
Processing: 173
Processing: 174
Processing: 175
Processing: 176
Processing: 177
Processing: 178
Processing: 179
Processing: 180
Processing: 181
Processing: 182
Processing: 183
Processing: 184
Processing: 185
Processing: 186
Processing: 187
Processing: 188
Processing: 189
Processing: 190
Processing: 191
Processing: 192
Processing: 193
Processing: 194
Processing: 195
Processing: 196
Processing: 197
Processing: 198
Processing: 199
Processing: 200
Processing: 201
Processing: 202
Processing: 203
Processing: 204
Processing: 205
Processing: 206
Processing: 207
Processing: 208
Processing: 209
Processing: 210
Processing: 211
Processing: 212
Processing: 213
Processing: 214
Processing: 215
Processing: 216
Processing: 217
Processing: 218
Processing: 219
Processing: 220
Processing: 221
Processing: 222
Processing: 223
Processing: 224
Processing: 225
Processing: 226
Processing: 227
Processing: 228
Processing: 229
Processing: 230
Processing: 231
Processi

...

Processing: 4973
Processing: 4974
Processing: 4975
Processing: 4976
Processing: 4977
Processing: 4978
Processing: 4979
Processing: 4980
Processing: 4981
Processing: 4982
Processing: 4983
Processing: 4984
Processing: 4985
Processing: 4986
Processing: 4987
Processing: 4988
Processing: 4989
Processing: 4990
Processing: 4991
Processing: 4992
Processing: 4993
Processing: 4994
Processing: 4995
Processing: 4996
Processing: 4997
Processing: 4998
Processing: 4999
Processing: 5000
Processing: 5001
Processing: 5002
Processing: 5003
Processing: 5004
Processing: 5005
Processing: 5006
Processing: 5007
Processing: 5008
Processing: 5009
Processing: 5010
Processing: 5011
Processing: 5012
Processing: 5013
Processing: 5014
Processing: 5015
Processing: 5016
Processing: 5017
Processing: 5018
Processing: 5019
Processing: 5020
Processing: 5021
Processing: 5022
Processing: 5023
Processing: 5024
Processing: 5025
Processing: 5026
Processing: 5027
Processing: 5028
Processing: 5029
Processing: 5030
Processing: 50

[21:24:32] 3mof_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.


Processing: 5099
Processing: 5100
Processing: 5101
Processing: 5102
Processing: 5103
Processing: 5104
Processing: 5105
Processing: 5106
Processing: 5107
Processing: 5108
Processing: 5109
Processing: 5110
Processing: 5111
Processing: 5112
Processing: 5113
Processing: 5114
Processing: 5115
Processing: 5116
Processing: 5117
Processing: 5118
Processing: 5119
Processing: 5120
Processing: 5121
Processing: 5122
Processing: 5123
Processing: 5124
Processing: 5125
Processing: 5126
Processing: 5127
Processing: 5128
Processing: 5129
Processing: 5130
Processing: 5131
Processing: 5132
Processing: 5133
Processing: 5134
Processing: 5135
Processing: 5136
Processing: 5137
Processing: 5138
Processing: 5139
Processing: 5140
Processing: 5141
Processing: 5142
Processing: 5143
Processing: 5144
Processing: 5145
Processing: 5146
Processing: 5147
Processing: 5148
Processing: 5149
Processing: 5150
Processing: 5151
Processing: 5152
Processing: 5153
Processing: 5154
Processing: 5155
Processing: 5156
Processing: 51

[21:34:22] Explicit valence for atom # 388 O, 3, is greater than permitted


Error processing 2zjw: Failed to read PDB file: /content/PDBbind_v2020_refined/refined-set/2zjw/2zjw_protein.pdb
Processing: 5235
Processing: 5236
Processing: 5237
Processing: 5238
Processing: 5239
Processing: 5240
Processing: 5241
Processing: 5242
Processing: 5243
Processing: 5244
Processing: 5245
Processing: 5246
Processing: 5247
Processing: 5248
Processing: 5249
Processing: 5250
Processing: 5251
Processing: 5252
Processing: 5253
Processing: 5254
Processing: 5255
Processing: 5256
Processing: 5257
Processing: 5258
Processing: 5259
Processing: 5260
Processing: 5261
Processed 5261 receptor-ligand pairs.
Skipped 55 due to errors.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GraphModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = global_mean_pool(x, data.batch)
        x = self.fc(x)
        return x

class BindingAffinityModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BindingAffinityModel, self).__init__()
        self.receptor_model = GraphModel(input_dim, hidden_dim, output_dim)
        self.ligand_model = GraphModel(input_dim, hidden_dim, output_dim)
        self.fc = nn.Linear(2 * output_dim, 1)

    def forward(self, receptor_data, ligand_data):
        receptor_embedding = self.receptor_model(receptor_data)
        ligand_embedding = self.ligand_model(ligand_data)
        x = torch.cat([receptor_embedding, ligand_embedding], dim=1)
        x = self.fc(x)
        return x

input_dim = 1
hidden_dim = 64
output_dim = 128
model = BindingAffinityModel(input_dim, hidden_dim, output_dim)

In [ ]:
import torch.optim as optim
from torch_geometric.data import DataLoader
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data


graph_data_list = []

for i, (pdb_id, receptor_graph, ligand_graph) in enumerate(graph_data):
    receptor_data = Data(x=receptor_graph['x'], edge_index=receptor_graph['edge_index'], edge_attr=receptor_graph['edge_attr'])
    ligand_data = Data(x=ligand_graph['x'], edge_index=ligand_graph['edge_index'], edge_attr=ligand_graph['edge_attr'])
    
    if pdb_id in binding_affinities_dict:
      binding_affinity = torch.tensor([binding_affinities_dict[pdb_id]], dtype=torch.float32)

    data = {
        'receptor_graph': receptor_data,
        'ligand_graph': ligand_data,
        'binding_affinity': binding_affinity
    }

    graph_data_list.append(data)


# 1. Разделите датасет на обучающую и тестовую выборки
train_data, test_data = train_test_split(graph_data_list, test_size=0.2, random_state=42)

# 2. Создайте DataLoader для обучающей и тестовой выборок
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

#batch_size = 32
#data_loader = DataLoader(graph_data_list, batch_size=batch_size, shuffle=True)



# 3. Задайте функцию потерь и оптимизатор
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Обучите модель на обучающей выборке
num_epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        receptor_data, ligand_data = batch['receptor_graph'].to(device), batch['ligand_graph'].to(device)
        target = batch['binding_affinity'].to(device)

        optimizer.zero_grad()
        output = model(receptor_data, ligand_data)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# 5. Оцените модель на тестовой выборке
model.eval()
test_loss = 0

with torch.no_grad():
    for batch in test_loader:
        receptor_data, ligand_data = batch['receptor_graph'].to(device), batch['ligand_graph'].to(device)
        target = batch['binding_affinity'].to(device)

        output = model(receptor_data, ligand_data)
        loss = criterion(output.squeeze(), target)

        test_loss += loss.item()

print(f"Test Loss: {test_loss / len(test_loader)}")

/usr/local/lib/python3.9/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([48, 1])) that is different to the input size (torch.Size([48])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 19006072676.848484


KeyboardInterrupt: ignored

In [ ]:
# Загрузите файлы рецептора и лиганда
receptor_path = "BACE1.pdb"
ligand_path = "Berberin.mol2"

# Преобразуйте файлы в графовое представление
receptor_mol = read_receptor_pdb(receptor_path)
ligand_mol = read_ligand_mol2(ligand_path)

receptor_graph = mol_to_graph(receptor_mol)
ligand_graph = mol_to_graph(ligand_mol)

# Создайте объекты Data для рецептора и лиганда
receptor_data = Data(x=receptor_graph['x'], edge_index=receptor_graph['edge_index'], edge_attr=receptor_graph['edge_attr'])
ligand_data = Data(x=ligand_graph['x'], edge_index=ligand_graph['edge_index'], edge_attr=ligand_graph['edge_attr'])

# Включите режим оценки для модели
#model.eval()

# Выполните прямой проход с данными рецептора и лиганда
#with torch.no_grad():
#    receptor_data = receptor_data.to(device)
#    ligand_data = ligand_data.to(device)
#    output = model(receptor_data, ligand_data)

# Интерпретируйте результаты
#predicted_binding_affinity = output.item()
#print(f"Predicted binding affinity: {predicted_binding_affinity}")

In [ ]:
# Включите режим оценки для модели
model.eval()

# Выполните прямой проход с данными рецептора и лиганда
with torch.no_grad():
    receptor_data = receptor_data.to(device)
    ligand_data = ligand_data.to(device)
    output = model(receptor_data, ligand_data)

# Интерпретируйте результаты
predicted_binding_affinity = output.item()
print(f"Predicted binding affinity: {predicted_binding_affinity}")

Predicted binding affinity: 170579.390625


In [ ]:
# Включите режим оценки для модели
model.eval()

# Выполните прямой проход с данными рецептора и лиганда
with torch.no_grad():
    receptor_data = receptor_data.to(device)
    ligand_data = ligand_data.to(device)
    output = model(receptor_data, ligand_data)

# Интерпретируйте результаты
predicted_binding_affinity = output.item()
predicted_binding_affinity = scaler.inverse_transform(np.array(predicted_binding_affinity).reshape(-1,1))
predicted_binding_affinity = predicted_binding_affinity[0][0]
print(f"Predicted binding affinity: {predicted_binding_affinity}")

ValueError: ignored

In [ ]:
import pickle

with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('model.pickle', 'rb') as f:
    model = pickle.load(f)

In [ ]:
# Задайте путь для сохранения graph_data_list
graph_data_list_path = "graph_data.pkl"

# Сохраните graph_data_list на диск
with open(graph_data_list_path, 'wb') as file:
    pickle.dump(graph_data, file)

In [ ]:
# Загрузите graph_data_list с диска
with open(graph_data_list_path, 'rb') as file:
    graph_data = pickle.load(file)
